In [1]:
import json
import sys
import wikipedia

sys.path.append('..')
from tools.TextProcessing import build_word_tree, process_keywords
from tools.BasicUtils import my_write, my_csv_read, MultiThreading, get_wikipedia_entity

<h2> Select keyword file and generate keyword list

In [ ]:
# unknown keyword file for now
stable_kw = []
unstable_kw = []
f_in = my_csv_read('../data/raw_data/keyword.csv', delimiter='\t')
keywords = [line[0] for line in f_in]

In [ ]:
# Collect keywords that can be linked to a wikipedia page
stable_kw = []
unstable_kw = []
r = my_csv_read('../data/raw_data/terms-cs-cfl-epoch200.txt', delimiter='\t')
candidate_kw_list = [item[0] for item in r]
stable_kw, unstable_kw = process_keywords(candidate_kw_list)

mt = MultiThreading()
ret = mt.run(lambda x: (x, get_wikipedia_entity(x)), stable_kw, thread_num=2)

<h2> Process keyword list and generate word tree

In [ ]:
# Process keyword list and store the results
stable_kw, unstable_kw = process_keywords(keywords)
my_write('../data/corpus/keyword_f.txt', stable_kw)
my_write('../data/temp/unstable_keyword.txt', unstable_kw)
# Generate word tree (25 seconds)
build_word_tree('../data/corpus/keyword_f.txt', '../data/corpus/wordtree.json', '../data/corpus/entity.txt')

In [ ]:
# Check stable keywords
'-' in stable_kw

<h2> Process sentences

In [ ]:
# Extract sentence from small corpus (2 minute)
j = json.load(open('../data/raw_data/small_arxiv.json', 'r'))
sents = [i['abstract'].replace('\n', ' ').strip() for i in j]
with open('../data/corpus/pre_small_sent.txt', 'w') as f_out:
    f_out.write('\n'.join(sents))

!sed 's/\$//g;s/\\/ /g;s/---*/, /g;s/([^)]*)//g;s/{[^)]*}//g;s/-/ - /g' ../data/corpus/pre_small_sent.txt | tr -s [:space:] | tr '[:upper:]' '[:lower:]' > ../data/corpus/small_sent.txt

!python ../py/sent_tokenize.py ../data/corpus/small_sent.txt ../data/corpus/small_sent.txt

!rm ../data/corpus/pre_small_sent.txt

In [ ]:
# Extract sentence from 1st_sents_new.json
file_dir = '../data/raw_data/1st-sents-new.json'

j = json.load(open(file_dir, 'r'))
sents = [i['sentence'].strip() for i in j.values()]
with open('../data/corpus/pre_1st_sent.txt', 'w') as f_out:
    f_out.write('\n'.join(sents))

!sed 's/\$//g;s/\\/ /g;s/---*/, /g;s/([^)]*)//g;s/{[^)]*}//g;s/-/ - /g' ../data/corpus/pre_1st_sent.txt | tr -s [:space:] | tr '[:upper:]' '[:lower:]' > ../data/corpus/1st_sent.txt

!rm ../data/corpus/pre_1st_sent.txt

In [ ]:
# Generate Reformed sentences (3 minutes)
# To run the code in the backend, use the gen_reform.py in the "py" folder

## Keyword process using terms-cs-cfl-epoch200.txt and 1st-sents-new.json

In [2]:
wiki_kw_dict = json.load(open('../data/corpus/1st-sents-lowercase.json', 'r'))

True


In [12]:
wiki_kw_dict['python']

{'sentence': 'python was a cold war contingency plan of the british government for the continuity of government in the event of nuclear war .',
 'entity': {'cold war': 'cold war',
  'british government': 'government of the united kingdom',
  'continuity of government': 'continuity of government',
  'nuclear war': 'nuclear warfare'}}

In [20]:
'convolutional neural network' in candidate_kw_list

True

In [21]:
with open('../data/corpus/keyword_f.txt', 'w') as f_out:
    f_out.write('\n'.join(stable_kw))